# Flight Data Collector

## Imports

In [1]:
import requests
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime, timezone, timedelta
import time

### opensky_api module setup

In [2]:
from opensky_api import OpenSkyApi
from config import opensky_user, opensky_pass

## Supporting Functions

In [3]:
def datetime_to_unix(dt):
    return int(time.mktime(dt.timetuple()))

## Connect to OpenSky API

In [5]:
api = OpenSkyApi(opensky_user, opensky_pass)

## Collect Flight Data and Trajectories

In [6]:
# collect unique ICAO transponder addresses per airport

airports = ['KMSP']

now = datetime.now()
one_hour_ago = now - timedelta(hours=1)
print(now)
print(one_hour_ago)

start_ts = datetime_to_unix(one_hour_ago)
end_ts = datetime_to_unix(now)
print(start_ts)
print(end_ts)

unique_icao_addresses = []

for airport in airports:
    arrivals = api.get_arrivals_by_airport(airport, start_ts, end_ts)
    departures = api.get_departures_by_airport(airport, start_ts, end_ts)

    # get unique arrival aircraft
    if arrivals is not None:
        for flight in arrivals:
            if flight.icao24 not in unique_icao_addresses:
                unique_icao_addresses.append(flight.icao24)

    # get unique departure aircraft
    if departures is not None:
        for flight in departures:
            if flight.icao24 not in unique_icao_addresses:
                unique_icao_addresses.append(flight.icao24)

print()
print('Unique aircraft identifiers arriving or departing MSP')
print(unique_icao_addresses)

2024-07-18 10:54:17.746245
2024-07-18 09:54:17.746245
1721314457
1721318057

Unique aircraft identifiers arriving or departing MSP
['a1863b', 'a3ed69', 'acd2d3', 'acaf0b', 'a25cdc', 'a3f4d7', 'a4077d', 'a39ac7', 'a41144', 'a8fd5a', 'a7e18a', 'aa60da', 'a33afa', 'abc91f', 'ade548', 'adc180', 'abbf49', 'ac2c9c', 'a39723', 'a2355d', 'a359bc', 'ad4395', 'a7d2ae', 'a369be', 'ac1466', 'abe1c2', 'abe3c8', 'a022c6', 'a305fd', 'a39d95', 'abf09e', 'a34dbf', 'a37aa3', 'ac85ce']


### Collect flight data

Returns 'FlightData' type:

Class that represents data of certain flight. It has the following fields:

- icao24: str - Unique ICAO 24-bit address of the transponder in hex string representation. All letters are lower case.
- firstSeen: int - Estimated time of departure for the flight as Unix time (seconds since epoch).
estDepartureAirport: str - ICAO code of the estimated departure airport. Can be null if the airport could not be identified.
- lastSeen: int - Estimated time of arrival for the flight as Unix time (seconds since epoch).
estArrivalAirport: str - ICAO code of the estimated arrival airport. Can be null if the airport could not be identified.
- callsign: str - Callsign of the vehicle (8 chars). Can be null if no callsign has been received. If the vehicle transmits multiple callsigns during the flight, we take the one seen most frequently.
- estDepartureAirportHorizDistance: int - Horizontal distance of the last received airborne position to the estimated departure airport in meters.
- estDepartureAirportVertDistance: int - Vertical distance of the last received airborne position to the estimated departure airport in meters.
- estArrivalAirportHorizDistance: int - Horizontal distance of the last received airborne position to the estimated arrival airport in meters.
- estArrivalAirportVertDistance: int - Vertical distance of the last received airborne position to the estimated arrival airport in meters.
- departureAirportCandidatesCount: int - Number of other possible departure airports. These are airports in short distance to estDepartureAirport.
- arrivalAirportCandidatesCount: int - Number of other possible departure airports.


In [55]:
# collect flight data for each aircraft (by unique ICAO transponder address)

# temporary - limit search by two aircraft
unique_icao_addresses = ['a65092', 'a327a4']

for icao in unique_icao_addresses:
    flight_data = api.get_flights_by_aircraft(icao, start_ts, end_ts)
    print(flight_data)

None
None


### Collect state vectors

Returns 'StateVector' type:

Represents the state of a vehicle at a particular time. It has the following fields:

- icao24: str - ICAO24 address of the transmitter in hex string representation.
- callsign: str - callsign of the vehicle. Can be None if no callsign has been received.
- origin_country: str - inferred through the ICAO24 address.
- time_position: int - seconds since epoch of last position report. Can be None if there was no position report received by OpenSky within 15s before.
- last_contact: int - seconds since epoch of last received message from this transponder.
- longitude: float - in ellipsoidal coordinates (WGS-84) and degrees. Can be None.
- latitude: float - in ellipsoidal coordinates (WGS-84) and degrees. Can be None.
- geo_altitude: float - geometric altitude in meters. Can be None.
- on_ground: bool - true if aircraft is on ground (sends ADS-B surface position reports).
- velocity: float - over ground in m/s. Can be None if information not present.
- true_track: float - in decimal degrees (0 is north). Can be None if information not present.
- vertical_rate: float - in m/s, incline is positive, decline negative. Can be None if information not present.
- sensors: list [int] - serial numbers of sensors which received messages from the vehicle within the validity period of this state vector. Can be None if no filtering for sensor has been requested.
- baro_altitude: float - barometric altitude in meters. Can be None.
- squawk: str - transponder code aka Squawk. Can be None.
- spi: bool - special purpose indicator.
- position_source: int - origin of this state’s position: 0 = ADS-B, 1 = ASTERIX, 2 = MLAT, 3 = FLARM
- category: int - aircraft category: 0 = No information at all, 1 = No ADS-B Emitter Category Information, 2 = Light (< 15500 lbs), 3 = Small (15500 to 75000 lbs), 4 = Large (75000 to 300000 lbs), 5 = High Vortex Large (aircraft such as B-757), 6 = Heavy (> 300000 lbs), 7 = High Performance (> 5g acceleration and 400 kts), 8 = Rotorcraft, 9 = Glider / sailplane, 10 = Lighter-than-air, 11 = Parachutist / Skydiver, 12 = Ultralight / hang-glider / paraglider, 13 = Reserved, 14 = Unmanned Aerial Vehicle, 15 = Space / Trans-atmospheric vehicle, 16 = Surface Vehicle – Emergency Vehicle, 17 = Surface Vehicle – Service Vehicle, 18 = Point Obstacle (includes tethered balloons), 19 = Cluster Obstacle, 20 = Line Obstacle.


In [53]:
# collect "state vectors" for each aircraft (by unique ICAO transponder address)

# temporary - limit search by two aircraft
unique_icao_addresses = ['a65092', 'a327a4']

for icao in unique_icao_addresses:
    state_data = api.get_states(icao24=icao)
    print(state_data)

{   'states': [   StateVector(dict_values(['a65092', 'DAL1484 ', 'United States', 1721182737, 1721182737, -93.0946, 44.9517, 3177.54, False, 167.98, 120.96, 5.85, None, 3246.12, None, False, 0, 0]))],
    'time': 1721182737}
None


### Collect trajectory data

Returns both 'FlightTrack' and 'Waypoint' data types:

FlightTrack
Class that represents the trajectory for a certain aircraft at a given time.:
- icao24: str - Unique ICAO 24-bit address of the transponder in lower case hex string representation.
- startTime: int - Time of the first waypoint in seconds since epoch (Unix time).
- endTime: int - Time of the last waypoint in seconds since epoch (Unix time).
- calllsign: str - Callsign (8 characters) that holds for the whole track. Can be null.
- path: list [Waypoint] - waypoints of the trajectory.

Waypoint
Class that represents the single waypoint that is a basic part of flight trajectory:
- time: int - Time which the given waypoint is associated with in seconds since epoch (Unix time).
- latitude: float - WGS-84 latitude in decimal degrees. Can be null.
- longitude: float - WGS-84 longitude in decimal degrees. Can be null.
- baro_altitude: float - Barometric altitude in meters. Can be null.
- true_track: float - True track in decimal degrees clockwise from north (north=0°). Can be null.
- on_ground: bool - Boolean value which indicates if the position was retrieved from a surface position report.

In [54]:
# collect trajectory for each aircraft (by unique ICAO transponder address)

# temporary - limit search by two aircraft
unique_icao_addresses = ['a65092', 'a327a4']

for icao in unique_icao_addresses:
    track_data = api.get_track_by_aircraft(icao24=icao)
    print(track_data)

{   'callsign': 'DAL1484 ',
    'endTime': 1721182840.0,
    'icao24': 'a65092',
    'path': [   [1721182494, 44.8764, -93.2079, 0, 301, False],
                [1721182510, 44.8819, -93.2205, 0, 301, False],
                [1721182524, 44.8875, -93.2338, 304, 300, False],
                [1721182554, 44.8996, -93.2637, 609, 300, False],
                [1721182559, 44.9023, -93.2697, 609, 304, False],
                [1721182564, 44.9055, -93.2751, 914, 312, False],
                [1721182569, 44.9089, -93.2794, 914, 321, False],
                [1721182574, 44.9135, -93.2834, 914, 332, False],
                [1721182579, 44.9187, -93.2862, 914, 342, False],
                [1721182584, 44.924, -93.2877, 914, 352, False],
                [1721182589, 44.9299, -93.2883, 914, 358, False],
                [1721182594, 44.9356, -93.2881, 914, 2, False],
                [1721182608, 44.9521, -93.2868, 1219, 3, False],
                [1721182613, 44.9583, -93.2862, 1219, 4, False],
    